# Standard Bank Tech Impact Challenge: Xente credit scoring challenge 

In this challenge, the task is to predict from a historical dataset of transactions and repayments the likelihood of default in each transaction in a test set contaning various customers.
This is a binary classification problem where the output should be 1 for a defaulted loan and 0 otherwise.

The evaluation metric for this challenge is the Area Under the Curve (AUC).

In [1]:
## importing libraries
import pandas as pd
import numpy as np

In [2]:
## reading the files and loading them into dataframes.
train = pd.read_csv('C:/Users/Amine/Desktop/Standard Bank Tech Impact Challenge Xente credit scoring challenge/Train.csv')
test= pd.read_csv('C:/Users/Amine/Desktop/Standard Bank Tech Impact Challenge Xente credit scoring challenge/Test.csv')
sample = pd.read_csv('C:/Users/Amine/Desktop/Standard Bank Tech Impact Challenge Xente credit scoring challenge/sample_submission.csv')
mask = pd.read_csv('C:/Users/Amine/Desktop/Standard Bank Tech Impact Challenge Xente credit scoring challenge/unlinked_masked_final.csv')
variabs = pd.read_csv('C:/Users/Amine/Desktop/Standard Bank Tech Impact Challenge Xente credit scoring challenge/VariableDefinitions.csv')

I'm still not quite familiar with the nature of the data, and how data needs to be organized in order to properly model for this challenge. I will update this notebook as soon as i get better with this particular problem.

###### Timeline of the data

In [3]:
## The data was ordered by date. So lets check it.
train.head(3)

,CustomerId,TransactionStartTime,Value,Amount,TransactionId,BatchId,SubscriptionId,CurrencyCode,CountryCode,ProviderId,...,LoanId,PaidOnDate,IsFinalPayBack,InvestorId,DueDate,LoanApplicationId,PayBackId,ThirdPartyId,IsThirdPartyConfirmed,IsDefaulted
0,CustomerId_27,2018-09-21 12:17:39,550.0,-550.0,TransactionId_1683,BatchId_641,SubscriptionId_2,UGX,256,ProviderId_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CustomerId_27,2018-09-25 09:20:29,550.0,-550.0,TransactionId_2235,BatchId_820,SubscriptionId_2,UGX,256,ProviderId_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CustomerId_27,2018-09-25 10:33:31,550.0,-550.0,TransactionId_1053,BatchId_210,SubscriptionId_4,UGX,256,ProviderId_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


So it starts from 21th of September 2018 **( Do you remember 21st night of September? Customer_27 does. Couldn't help it sorry)**

In [4]:
## It ends 31th of March 2019
train.tail(3)

,CustomerId,TransactionStartTime,Value,Amount,TransactionId,BatchId,SubscriptionId,CurrencyCode,CountryCode,ProviderId,...,LoanId,PaidOnDate,IsFinalPayBack,InvestorId,DueDate,LoanApplicationId,PayBackId,ThirdPartyId,IsThirdPartyConfirmed,IsDefaulted
2097,CustomerId_34,2019-03-31 05:40:10,5000.0,-5000.0,TransactionId_2507,BatchId_1093,SubscriptionId_7,UGX,256,ProviderId_1,...,LoanId_1585,2019-06-15 06:20:05,1.0,InvestorId_1,2019-04-30 05:40:02,LoanApplicationId_1466,PayBackId_1271,ThirdPartyId_1981,1.0,0.0
2098,CustomerId_346,2019-03-31 09:37:19,1500.0,-1500.0,TransactionId_540,BatchId_1239,SubscriptionId_7,UGX,256,ProviderId_1,...,LoanId_1314,2019-04-23 15:44:15,1.0,InvestorId_1,2019-04-30 09:37:15,LoanApplicationId_1389,PayBackId_945,ThirdPartyId_705,1.0,0.0
2099,CustomerId_188,2019-03-31 12:16:33,1000.0,-1000.0,TransactionId_1889,BatchId_355,SubscriptionId_7,UGX,256,ProviderId_1,...,LoanId_1570,2019-04-10 17:33:19,1.0,InvestorId_1,2019-04-30 12:16:26,LoanApplicationId_57,PayBackId_830,ThirdPartyId_899,1.0,0.0


It says on the challenge data section that the maximum date is 17 July 2019. So we can logically assume that we're being tested on transactions between 01-04-2019 and 17-07-2019

In [5]:
## Lets check that
print(test.TransactionStartTime.min(),test.TransactionStartTime.max())

2019-03-31 13:33:05 2019-07-17 05:34:45


Actually the testing starts the same day train dates end. The 31th of March.

###### What's a loan? And what's loan default?

Loan default occurs when a borrower fails to pay back a debt according to the initial arrangement. 
In the case of most consumer loans, this means that successive payments have been missed over the course of weeks or months. 
Fortunately, lenders and loan servicers usually allow a grace period before penalizing the borrower after missing one payment. 
The period between missing a loan payment and having the loan default is known as delinquency. 
The delinquency period gives the debtor time to avoid default by contacting their loan servicer or making up missed payments.

So there's a grace period in which the date on which the borrower is set to pay the loan or rather a portion of it is exceeded. The loan doesn't default immediately, that period is called deliquency. It should be a feature to extract and would be a good one logically. If someone is not defaulting on loans, but is missing a lot of payments he should be more likely to default soon.

**In the context of this competition and the nature of the loans granted, the grace period is 30 days!**

In [6]:
## lets see if the grace period of 30 days is true for this data
train[(train.PaidOnDate>train.DueDate)].IsDefaulted.value_counts()

0.0    341
1.0     75
Name: IsDefaulted, dtype: int64

**Something is fishy.. Not all the transactions marked as Defaulted are in this dataframe. Is it possible to have transactions that were paid before the duedate and are still marked as Defaulted? lets explore that**

In [7]:
len(train[(train.PaidOnDate<train.DueDate)&(train.IsDefaulted==1)])

103

In [8]:
## Taking the example of the LoanId_1376
train[train.LoanId=='LoanId_1376']

,CustomerId,TransactionStartTime,Value,Amount,TransactionId,BatchId,SubscriptionId,CurrencyCode,CountryCode,ProviderId,...,LoanId,PaidOnDate,IsFinalPayBack,InvestorId,DueDate,LoanApplicationId,PayBackId,ThirdPartyId,IsThirdPartyConfirmed,IsDefaulted
1847,CustomerId_158,2019-03-07 08:16:11,192000.0,-192000.0,TransactionId_2374,BatchId_1636,SubscriptionId_6,UGX,256,ProviderId_1,...,LoanId_1376,2019-04-02 10:04:20,0.0,InvestorId_2,2019-04-06 08:16:07,LoanApplicationId_323,PayBackId_449,ThirdPartyId_216,1.0,1.0
1848,CustomerId_158,2019-03-07 08:16:11,192000.0,-192000.0,TransactionId_2374,BatchId_1636,SubscriptionId_6,UGX,256,ProviderId_1,...,LoanId_1376,2019-04-02 20:25:15,0.0,InvestorId_2,2019-04-06 08:16:07,LoanApplicationId_323,PayBackId_843,ThirdPartyId_169,1.0,1.0
1849,CustomerId_158,2019-03-07 08:16:11,192000.0,-192000.0,TransactionId_2374,BatchId_1636,SubscriptionId_6,UGX,256,ProviderId_1,...,LoanId_1376,2019-05-20 08:15:05,0.0,InvestorId_2,2019-04-06 08:16:07,LoanApplicationId_323,PayBackId_1533,ThirdPartyId_1915,1.0,1.0
1850,CustomerId_158,2019-03-07 08:16:11,192000.0,-192000.0,TransactionId_2374,BatchId_1636,SubscriptionId_6,UGX,256,ProviderId_1,...,LoanId_1376,2019-05-28 11:05:04,0.0,InvestorId_2,2019-04-06 08:16:07,LoanApplicationId_323,PayBackId_443,ThirdPartyId_460,1.0,1.0


***Alright, i can make some sense out of it. Since a single loan is paid by multiple transactions, when a loan defaults, all the transactions associated with that loan are marked as Defaulted. In the example above, you can see some payments ( PaidOnDate ) were made sooner than the DueDate but 2 associated payments were made much later which made the whole loan default. And all transactions associated to that loan are defaulted.***

In [9]:
multiloans=train.groupby('LoanId').size()[train.groupby('LoanId').size()>1].index

In [10]:
train[train.LoanId.isin(multiloans)].LoanId.nunique()

150

In [11]:
train[train.LoanId.isin(multiloans)].TransactionId.nunique()

151

**Interesting .. The number of transactions is almost equal to the number of loans. Each data loan representation in this dataset is across one or multiple rows where supposedly a loan is paid using multiple transactions, but concretely here, almost each loan is represented by a single transaction.**

**PS : A single transaction means the same transaction ID eg: 4 rows with different times but the same TransactionId is a single transaction made across 4 times**

###### Loan Application Refusal aka the Nan rows in the loan part of the dataset.

In [12]:
rejectedcustomers=train[train.TransactionStatus==0]['CustomerId'].unique()

Common knowledge by a doing a quick Google search shows that the most common reason for loan application rejection is the borrowing and repaying history. If a person tends to miss payments on a loan, they get a bad 'credit score' which is a sort of a credibility score. Extracting features that describe the customer behavior here from past loans is bound to be useful and necessary for any model that tries to solve this kind of problem.

In [13]:
train[(train.IsDefaulted==1)&(train.CustomerId.isin(rejectedcustomers))].CustomerId.unique()

array(['CustomerId_474', 'CustomerId_298', 'CustomerId_501',
       'CustomerId_350', 'CustomerId_222', 'CustomerId_223'], dtype=object)

So from the rejected customers for loan applications, only 6 customers defaulted before and then re-applied again for loans only to get rejected. Now something else intrigues me.. Are there customers that defaulted but were still successful in getting loans after that? lets check.

In [14]:
alldefaultedcustoms=train[train.IsDefaulted==1].CustomerId.unique()

In [15]:
numberofloandsperrejectedcustomer=train[train.CustomerId.isin(alldefaultedcustoms)].groupby('CustomerId').LoanId.nunique()

In [16]:
numberofloandsperrejectedcustomer[numberofloandsperrejectedcustomer>1]
manycusts=numberofloandsperrejectedcustomer[numberofloandsperrejectedcustomer>1].index

Ok, these are customers that were granted more than one loan in their history an defaulted on at least one of their loans at some point in time. All is left is to check if anyone of them has a loan accepted after defaulting on a prior one.

In [17]:
train[train.CustomerId.isin(manycusts)].groupby(['CustomerId','LoanId'],as_index=False).size()

CustomerId      LoanId     
CustomerId_120  LoanId_1099    1
                LoanId_352     1
                LoanId_526     2
                LoanId_665     2
                LoanId_839     2
                LoanId_964     1
CustomerId_168  LoanId_1015    1
                LoanId_1115    1
                LoanId_1195    2
                LoanId_1412    4
                LoanId_152     2
                LoanId_201     1
                LoanId_256     1
                LoanId_402     1
                LoanId_594     2
                LoanId_621     2
                LoanId_914     1
CustomerId_223  LoanId_1263    2
                LoanId_641     1
CustomerId_263  LoanId_1283    1
                LoanId_149     1
                LoanId_1603    1
                LoanId_329     1
                LoanId_728     1
CustomerId_298  LoanId_1151    1
                LoanId_1568    1
                LoanId_419     1
CustomerId_350  LoanId_1069    1
                LoanId_1311    1
               

In [18]:
train[train.CustomerId=='CustomerId_120']

,CustomerId,TransactionStartTime,Value,Amount,TransactionId,BatchId,SubscriptionId,CurrencyCode,CountryCode,ProviderId,...,LoanId,PaidOnDate,IsFinalPayBack,InvestorId,DueDate,LoanApplicationId,PayBackId,ThirdPartyId,IsThirdPartyConfirmed,IsDefaulted
159,CustomerId_120,2018-11-26 20:23:21,2000.0,-2000.0,TransactionId_1510,BatchId_1504,SubscriptionId_7,UGX,256,ProviderId_1,...,LoanId_352,2018-12-07 17:36:20,1.0,InvestorId_1,2018-12-26 20:23:15,LoanApplicationId_1226,PayBackId_1116,ThirdPartyId_1278,1.0,0.0
468,CustomerId_120,2018-12-18 11:15:50,1000.0,-1000.0,TransactionId_2258,BatchId_2387,SubscriptionId_7,UGX,256,ProviderId_1,...,LoanId_1099,2018-12-28 17:26:53,1.0,InvestorId_1,2019-01-17 11:15:46,LoanApplicationId_834,PayBackId_714,ThirdPartyId_369,1.0,0.0
571,CustomerId_120,2018-12-28 17:29:36,5000.0,-5000.0,TransactionId_760,BatchId_2080,SubscriptionId_7,UGX,256,ProviderId_1,...,LoanId_526,2019-01-09 18:47:10,0.0,InvestorId_1,2019-01-27 17:29:27,LoanApplicationId_194,PayBackId_680,ThirdPartyId_1627,1.0,0.0
572,CustomerId_120,2018-12-28 17:29:36,5000.0,-5000.0,TransactionId_760,BatchId_2080,SubscriptionId_7,UGX,256,ProviderId_1,...,LoanId_526,2019-01-23 06:26:00,1.0,InvestorId_1,2019-01-27 17:29:27,LoanApplicationId_194,PayBackId_1409,ThirdPartyId_192,1.0,0.0
844,CustomerId_120,2019-01-25 20:52:12,5000.0,-5000.0,TransactionId_127,BatchId_437,SubscriptionId_7,UGX,256,ProviderId_1,...,LoanId_839,2019-01-30 14:10:53,0.0,InvestorId_1,2019-02-24 20:52:09,LoanApplicationId_479,PayBackId_1980,ThirdPartyId_1289,1.0,0.0
845,CustomerId_120,2019-01-25 20:52:12,5000.0,-5000.0,TransactionId_127,BatchId_437,SubscriptionId_7,UGX,256,ProviderId_1,...,LoanId_839,2019-02-02 19:05:31,1.0,InvestorId_1,2019-02-24 20:52:09,LoanApplicationId_479,PayBackId_914,ThirdPartyId_848,1.0,0.0
1009,CustomerId_120,2019-02-04 20:24:47,4000.0,-4000.0,TransactionId_2023,BatchId_758,SubscriptionId_7,UGX,256,ProviderId_1,...,LoanId_665,2019-02-19 04:09:49,0.0,InvestorId_1,2019-03-06 20:24:41,LoanApplicationId_421,PayBackId_330,ThirdPartyId_1026,1.0,1.0
1010,CustomerId_120,2019-02-04 20:24:47,4000.0,-4000.0,TransactionId_2023,BatchId_758,SubscriptionId_7,UGX,256,ProviderId_1,...,LoanId_665,2019-03-09 11:11:47,0.0,InvestorId_1,2019-03-06 20:24:41,LoanApplicationId_421,PayBackId_1656,ThirdPartyId_1524,1.0,1.0
1887,CustomerId_120,2019-03-09 19:49:20,2000.0,-2000.0,TransactionId_1857,BatchId_233,SubscriptionId_7,UGX,256,ProviderId_1,...,LoanId_964,2019-04-04 10:32:20,1.0,InvestorId_1,2019-04-08 19:49:16,LoanApplicationId_816,PayBackId_539,ThirdPartyId_13,1.0,0.0


We found an instance of a customer who has defaulted on a loan. and then was granted another loan that was paid on duedate. interesting.. This is certainly an outlier that differs from most cases but it's still an interesting discovery.

###### How should we model?

Looking at the test set, the model should predict for a transactionID the IsDefaulted column ( our target )

In [21]:
len(train)

2100

In [22]:
train.TransactionId.nunique()

1758

We have some duplicated TransactionId's , we need to make those rows made of the same TransactionId denser and turn them into 1 row without losing the information of the many rows. Maybe by creating features specific for multi-row transactions?

In [24]:
train.groupby('TransactionId').head(1).IsDefaulted.value_counts()

0.0    1082
1.0      71
Name: IsDefaulted, dtype: int64

By taking only unique transactions , we can have an idea on the number of loans present in the train set. On the data info, we have the number of loans issued for the whole timeline described in the train+test. By deducting the number of loans in the train set from the total number of loans issued, we can already know the number of loans issued for test set.

Number of loans in the test set : 1631 - 1153 = 478.

In [28]:
test.LoanApplicationId.nunique()

478

confirmed by none other than the data. :D Although it was simpler to just count the LoanApplicationId in test set, but it's better for you to be sure that what was said in the competition info matches your findings in the data.

###### multi-rows transactions

In [30]:
train.columns

Index(['CustomerId', 'TransactionStartTime', 'Value', 'Amount',
       'TransactionId', 'BatchId', 'SubscriptionId', 'CurrencyCode',
       'CountryCode', 'ProviderId', 'ProductId', 'ProductCategory',
       'ChannelId', 'TransactionStatus', 'IssuedDateLoan', 'AmountLoan',
       'Currency', 'LoanId', 'PaidOnDate', 'IsFinalPayBack', 'InvestorId',
       'DueDate', 'LoanApplicationId', 'PayBackId', 'ThirdPartyId',
       'IsThirdPartyConfirmed', 'IsDefaulted'],
      dtype='object')

In [44]:
train[train.duplicated(subset=['TransactionId'],keep=False)]

,CustomerId,TransactionStartTime,Value,Amount,TransactionId,BatchId,SubscriptionId,CurrencyCode,CountryCode,ProviderId,...,LoanId,PaidOnDate,IsFinalPayBack,InvestorId,DueDate,LoanApplicationId,PayBackId,ThirdPartyId,IsThirdPartyConfirmed,IsDefaulted
9,CustomerId_258,2018-10-18 16:11:55,10000.0,-10000.0,TransactionId_1041,BatchId_1970,SubscriptionId_4,UGX,256,ProviderId_1,...,LoanId_62,2018-10-22 09:13:17,1.0,InvestorId_3,2018-11-17 16:11:04,NaN,PayBackId_1791,ThirdPartyId_648,0.0,0.0
10,CustomerId_258,2018-10-18 16:11:55,10000.0,-10000.0,TransactionId_1041,BatchId_1970,SubscriptionId_4,UGX,256,ProviderId_1,...,LoanId_62,2018-10-23 07:34:35,1.0,InvestorId_3,2018-11-17 16:11:04,NaN,PayBackId_1067,ThirdPartyId_68,0.0,0.0
11,CustomerId_258,2018-10-18 16:11:55,10000.0,-10000.0,TransactionId_1041,BatchId_1970,SubscriptionId_4,UGX,256,ProviderId_1,...,LoanId_62,2018-10-25 08:58:07,1.0,InvestorId_3,2018-11-17 16:11:04,NaN,PayBackId_928,ThirdPartyId_1883,0.0,0.0
35,CustomerId_405,2018-11-01 16:12:41,53500.0,-50000.0,TransactionId_549,BatchId_2426,SubscriptionId_1,UGX,256,ProviderId_1,...,LoanId_77,2018-12-06 09:26:57,0.0,InvestorId_2,2018-12-01 16:12:37,LoanApplicationId_534,PayBackId_165,ThirdPartyId_1760,1.0,0.0
36,CustomerId_405,2018-11-01 16:12:41,53500.0,-50000.0,TransactionId_549,BatchId_2426,SubscriptionId_1,UGX,256,ProviderId_1,...,LoanId_77,2018-12-11 17:00:26,1.0,InvestorId_2,2018-12-01 16:12:37,LoanApplicationId_534,PayBackId_1908,ThirdPartyId_1716,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2031,CustomerId_259,2019-03-22 16:45:26,100000.0,-100000.0,TransactionId_370,BatchId_2290,SubscriptionId_1,UGX,256,ProviderId_1,...,LoanId_1225,2019-04-04 19:36:29,1.0,InvestorId_2,NaN,LoanApplicationId_813,PayBackId_1170,ThirdPartyId_1904,1.0,0.0
2037,CustomerId_348,2019-03-23 06:44:43,2000.0,-2000.0,TransactionId_45,BatchId_1452,SubscriptionId_7,UGX,256,ProviderId_1,...,LoanId_210,2019-04-11 15:00:25,0.0,InvestorId_1,2019-04-22 06:44:38,LoanApplicationId_1213,PayBackId_288,ThirdPartyId_1004,1.0,0.0
2038,CustomerId_348,2019-03-23 06:44:43,2000.0,-2000.0,TransactionId_45,BatchId_1452,SubscriptionId_7,UGX,256,ProviderId_1,...,LoanId_210,2019-04-26 19:58:58,1.0,InvestorId_1,2019-04-22 06:44:38,LoanApplicationId_1213,PayBackId_1974,ThirdPartyId_1438,1.0,0.0
2074,CustomerId_211,2019-03-28 17:26:51,5000.0,-5000.0,TransactionId_496,BatchId_2028,SubscriptionId_7,UGX,256,ProviderId_1,...,LoanId_123,2019-03-29 10:12:39,0.0,InvestorId_1,2019-04-27 17:26:47,LoanApplicationId_372,PayBackId_1787,ThirdPartyId_1237,1.0,0.0


Duplicated TransactionId rows show that the only column that mostly different for all rows is PaidOnDate. So these duplicated TransactionId's are describing the loan payment installments. If we can manage to transfer this info into columns, we can group them into one row for each transactionId which should make our modelling easier.

Quote from the data info : "The number of observations in the train data sets exceeds the number of transactions, as a result of some transactions being paid in split payments/instalments."

###### Grouping the multi-rows transactions without losing as much information as possible.

In [49]:
## creating variables to transfer the information contained in the rows of the same transaction.
train['Number_Of_Split_Payments'] = 0 ## this is a count on the number of payments on the same loan. It will take a 0 for singled-rowed transactions, 1+ for multi-row transacs.
train['Sum_Diff_Time_Payments'] = 0 ## I'm thinking of summing the delays between all payments made on a loan. It will take 0 for loans paid in a single time, 1+ for multiple payments on the same loan.
test['Number_Of_Split_Payments']=0
test['Sum_Diff_Time_Payments']=0

Creating only 2 features out of that information for now. There's probably more info to extract, like the difference in time between each payment without summing them, but we don't know the maximum  number of payments made on a loan which will define how many columns to add in order to cover all possibilites. And you could create a certain number for this particular dataset, and you come across a transaction with a number of payments that exceeds the number you've trained your model on which could result in an error or worse, in ignoring the exceeding payments which will make the prediction somewhat false.

In [55]:
train[train.duplicated(subset=['TransactionId'],keep=False)].groupby('TransactionId').agg('count')['Number_Of_Split_Payments']

TransactionId
TransactionId_1020    2
TransactionId_1036    2
TransactionId_1039    7
TransactionId_1041    3
TransactionId_1056    2
                     ..
TransactionId_926     2
TransactionId_927     2
TransactionId_929     4
TransactionId_943     2
TransactionId_968     2
Name: Number_Of_Split_Payments, Length: 158, dtype: int64

In [64]:
## creating the feature : number of split payments on a loan.
train['Number_Of_Split_Payments']=train['TransactionId'].map(train.groupby('TransactionId').agg('count')['Number_Of_Split_Payments'])
test['Number_Of_Split_Payments']=test['TransactionId'].map(test.groupby('TransactionId').agg('count')['Number_Of_Split_Payments'])

Next up, we need to extract the date from each row in the same transaction, and then deduct the date of the first payment from the date of the final payment to get the sum of differences between dates of split payments.

In [77]:
train[train.duplicated(subset=['TransactionId'],keep=False)].groupby('TransactionId').agg('max')['PaidOnDate']

TransactionId
TransactionId_1020    2019-03-12 11:16:46
TransactionId_1036    2019-05-31 13:28:24
TransactionId_1039    2019-06-28 23:29:01
TransactionId_1041    2018-10-25 08:58:07
TransactionId_1056    2019-01-17 08:50:42
                             ...         
TransactionId_926     2019-02-08 04:08:27
TransactionId_927                     NaN
TransactionId_929     2019-04-14 06:13:45
TransactionId_943     2019-04-11 07:45:49
TransactionId_968     2019-01-23 16:14:50
Name: PaidOnDate, Length: 158, dtype: object

In [100]:
train[train.duplicated(subset=['TransactionId'],keep=False)].head(5)

,CustomerId,TransactionStartTime,Value,Amount,TransactionId,BatchId,SubscriptionId,CurrencyCode,CountryCode,ProviderId,...,IsFinalPayBack,InvestorId,DueDate,LoanApplicationId,PayBackId,ThirdPartyId,IsThirdPartyConfirmed,IsDefaulted,Number_Of_Split_Payments,Sum_Diff_Time_Payments
9,CustomerId_258,2018-10-18 16:11:55,10000.0,-10000.0,TransactionId_1041,BatchId_1970,SubscriptionId_4,UGX,256,ProviderId_1,...,1.0,InvestorId_3,2018-11-17 16:11:04,NaN,PayBackId_1791,ThirdPartyId_648,0.0,0.0,3,0
10,CustomerId_258,2018-10-18 16:11:55,10000.0,-10000.0,TransactionId_1041,BatchId_1970,SubscriptionId_4,UGX,256,ProviderId_1,...,1.0,InvestorId_3,2018-11-17 16:11:04,NaN,PayBackId_1067,ThirdPartyId_68,0.0,0.0,3,0
11,CustomerId_258,2018-10-18 16:11:55,10000.0,-10000.0,TransactionId_1041,BatchId_1970,SubscriptionId_4,UGX,256,ProviderId_1,...,1.0,InvestorId_3,2018-11-17 16:11:04,NaN,PayBackId_928,ThirdPartyId_1883,0.0,0.0,3,0
35,CustomerId_405,2018-11-01 16:12:41,53500.0,-50000.0,TransactionId_549,BatchId_2426,SubscriptionId_1,UGX,256,ProviderId_1,...,0.0,InvestorId_2,2018-12-01 16:12:37,LoanApplicationId_534,PayBackId_165,ThirdPartyId_1760,1.0,0.0,2,0
36,CustomerId_405,2018-11-01 16:12:41,53500.0,-50000.0,TransactionId_549,BatchId_2426,SubscriptionId_1,UGX,256,ProviderId_1,...,1.0,InvestorId_2,2018-12-01 16:12:37,LoanApplicationId_534,PayBackId_1908,ThirdPartyId_1716,1.0,0.0,2,0


###### accidental finding : 2 transaction ID's have two different customers each..which should not be correct.

after some verification on the transactions associated to those customers, i think it's safer to drop both those transactions ID's in the context of this challenge. If this was going to be put in production environnment i'd keep the transactions, or maybe separate the customers into different transactions ID's that I'd create manually, but i'm not into artificially creating transaction ID's in the data so dropping is easier for me.

In [71]:
train[train.TransactionId=='TransactionId_927']

,CustomerId,TransactionStartTime,Value,Amount,TransactionId,BatchId,SubscriptionId,CurrencyCode,CountryCode,ProviderId,...,IsFinalPayBack,InvestorId,DueDate,LoanApplicationId,PayBackId,ThirdPartyId,IsThirdPartyConfirmed,IsDefaulted,Number_Of_Split_Payments,Sum_Diff_Time_Payments
911,CustomerId_13,2019-01-29 20:15:49,5000.0,-5000.0,TransactionId_927,BatchId_1384,SubscriptionId_7,UGX,256,ProviderId_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0
1413,CustomerId_15,2019-02-19 17:42:51,53500.0,-50000.0,TransactionId_927,BatchId_1384,SubscriptionId_7,UGX,256,ProviderId_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0


In [88]:
train.groupby('TransactionId').CustomerId.nunique()[train.groupby('TransactionId').CustomerId.nunique()>1]

TransactionId
TransactionId_703    2
TransactionId_927    2
Name: CustomerId, dtype: int64

In [105]:
train.drop(train[(train.TransactionId=='TransactionId_703')|((train.TransactionId=='TransactionId_927'))].index,axis=0,inplace=True)

###### back to creating the time difference feature

In [125]:
## this variable uses PaidOnDate variable, which is a target-related variable. So it's not available for the test data.
## However, this behavior can be tied to customers and then mapped to customers existing in the test set.
train.loc[:,'Sum_Diff_Time_Payments']=train['TransactionId'].map((pd.to_datetime(train[train.duplicated(subset=['TransactionId'],keep=False)].groupby('TransactionId').agg('max')['PaidOnDate'])-pd.to_datetime(train[train.duplicated(subset=['TransactionId'],keep=False)].groupby('TransactionId').agg('min')['PaidOnDate'])).astype('timedelta64[h]'))

**Idea for a feature** : We can sum the delays between first and last payment across all customers, and then compute the average delay a customer does. If he exists in the test set, it should help the model differientiate between 'good' and 'bad' customers.

In the same context, we could also create a feature that indicates if a particular customer has defaulted before, if he exists in the test set, it will be easier for the model to predict since defaulting on loans as a behavior seems crucial in predicting whether that customer will default again or not ( and that's a heuristic approach , we need to check that in the data )

**Lets transform train and test into single row transactions ( hoping we got enough information from the rows ) so we can move on to modelling**

In [145]:
## Lets drop the duplicate rows with the same transaction ID and keep the last one. (as in with the latest payment installment )
train.drop_duplicates(subset=['TransactionId'],keep='last',inplace=True)
test.drop_duplicates(subset=['TransactionId'],keep='last',inplace=True)

In [148]:
train.head(3)

,CustomerId,TransactionStartTime,Value,Amount,TransactionId,BatchId,SubscriptionId,CurrencyCode,CountryCode,ProviderId,...,IsFinalPayBack,InvestorId,DueDate,LoanApplicationId,PayBackId,ThirdPartyId,IsThirdPartyConfirmed,IsDefaulted,Number_Of_Split_Payments,Sum_Diff_Time_Payments
0,CustomerId_27,2018-09-21 12:17:39,550.0,-550.0,TransactionId_1683,BatchId_641,SubscriptionId_2,UGX,256,ProviderId_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
1,CustomerId_27,2018-09-25 09:20:29,550.0,-550.0,TransactionId_2235,BatchId_820,SubscriptionId_2,UGX,256,ProviderId_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
2,CustomerId_27,2018-09-25 10:33:31,550.0,-550.0,TransactionId_1053,BatchId_210,SubscriptionId_4,UGX,256,ProviderId_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN


We now have train and test sets containing single-row transactions, we can start exploring other aspect of the data.

###### Co-occuring customers between train and test

In [155]:
## Lets see if and who are the customers existing in both train and test.
## checking whether there are customers from test, that exist in train
print('%i unique shared customers between train and test '%test[test.CustomerId.isin(train.CustomerId)].CustomerId.nunique())

91 unique shared customers between train and test 


More than half the test set is made of customers already existing in the train set which makes customer-specific behavior features really important for this challenge. Also important to note that, almost 400 transactions are made by new customers that need to be classified using another type of features like general customer behavior or other.

In [156]:
## Any customers that exist only in train?
print('%i unique customers existing only in train '%train[~(train.CustomerId.isin(test.CustomerId))].CustomerId.nunique())

329 unique customers existing only in train 


In [163]:
## Any customers that exist only in test?
print('%i unique new customers existing only in test'%(test.CustomerId.nunique()-91))

83 unique new customers existing only in test
